In [7]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
import datetime

In [8]:
batch_size = 16
df = pd.read_csv('dataset.csv')
labels = df['Artifact Id']
label_counts = labels.value_counts()
filtered_labels = label_counts[(label_counts >= 5) & (label_counts <= 200)] # Remove "Command" label
filtered_labels_list = filtered_labels.index.tolist()
def tokenize_data(texts, tokenizer):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
model_path = "model_2024-12-03_16-37-16.pth"
model_path = "my_dict.pth"
# model_path = "best_model.pth"
# model_path = "model_avr.pth"

In [35]:
baseline_csv = "baseline.csv"
# baseline_csv = "test_data.csv"
output_csv = "baselineVSmodel_predictions.csv"

baseline_df = pd.read_csv(baseline_csv)

saved_dict = torch.load(model_path, weights_only=True)

_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels=len(filtered_labels_list)  
)
_model.load_state_dict(saved_dict)
_model.to(device)
_model.eval()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encodings = tokenize_data(baseline_df['Example Description'].tolist(), tokenizer)
dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'])
data_loader = DataLoader(dataset, sampler=SequentialSampler(dataset), batch_size=batch_size) 

predictions = []
with torch.no_grad():
    for batch in tqdm(data_loader, desc="Making Predictions"):
        input_ids, attention_mask = [item.to(device) for item in batch]
        outputs = _model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().tolist())

prediction_column_name = f"{model_path} prediction"

baseline_df[prediction_column_name] = [
    filtered_labels_list[pred] for pred in predictions
]

# Keep baseline predictions for comparison
# baseline_df['baseline Prediction'] = baseline_df['Prediction']

# Save the predictions to CSV
baseline_df[['Artifact Id', prediction_column_name]].to_csv(output_csv, index=False)
print(f"Predictions saved to {output_csv}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Making Predictions:   0%|          | 0/1 [00:00<?, ?it/s]

Predictions saved to baselineVSmodel_predictions.csv


In [36]:
true_labels = baseline_df['Artifact Id'].tolist()
true_labels = [label.replace('d3f:', '') for label in baseline_df['Artifact Id'].tolist()]

baseline_predictions = baseline_df['Prediction'].tolist()
model_predictions = baseline_df[prediction_column_name].tolist()
model_predictions = [label.replace('d3f:', '') for label in baseline_df[prediction_column_name].tolist()]

# Calculate F1 Score for baseline model (assuming the labels are categorical)
f1_baseline = f1_score(true_labels, baseline_predictions, average='weighted')
f1_model = f1_score(true_labels, model_predictions, average='weighted')

# Calculate accuracy for both models
accuracy_baseline = np.sum(np.array(baseline_predictions) == np.array(true_labels)) / len(true_labels) * 100
accuracy_model = np.sum(np.array(model_predictions) == np.array(true_labels)) / len(true_labels) * 100

# Print out the results
print(f"Baseline Model F1 Score (Weighted): {f1_baseline:.4f}")
print(f"Baseline Model Accuracy: {accuracy_baseline:.2f}%")
print(f"Model F1 Score (Weighted): {f1_model:.4f}")
print(f"Model Accuracy: {accuracy_model:.2f}%")

Baseline Model F1 Score (Weighted): 0.1667
Baseline Model Accuracy: 18.75%
Model F1 Score (Weighted): 0.1053
Model Accuracy: 25.00%
